# DoppelGanger Video Call Generative Artwork

: Meet your doppelganger

# DOWNLOAD E4E AND LOAD

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!rm -rf /content/encoder4editing

In [ ]:
!git clone https://github.com/omertov/encoder4editing.git /content/encoder4editing


In [ ]:
!cp "/content/drive/MyDrive/e4e_ffhq_encode.pt" "/content/encoder4editing/e4e_ffhq.pt"


In [ ]:
#Check if model is downloaded

!ls -lh /content/encoder4editing/e4e_ffhq.pt

In [ ]:
# Ensure Ninja is installed for PyTorch C++ extensions
!pip install ninja --quiet
import os
os.environ['TORCH_EXTENSIONS_DIR'] = '/root/.cache/torch_extensions'

# Optional: clear corrupted builds (just in case)
!rm -rf /root/.cache/torch_extensions


In [ ]:
from argparse import Namespace
import torch
import sys
sys.path.append('/content/encoder4editing')

from models.psp import pSp

# Load the checkpoint
ckpt_path = "/content/encoder4editing/e4e_ffhq.pt"
ckpt = torch.load(ckpt_path, map_location='cuda')
opts = ckpt['opts']
opts['checkpoint_path'] = ckpt_path
opts = Namespace(**opts)

# Initialize and load the network
net = pSp(opts)
net.eval().cuda()

print("✅ e4e model loaded!")


# optional: TEST THE MODEL UPLOAD A PHOTO OF YOUR FACE

In [ ]:
# Install face_recognition (quietly)
!pip install face_recognition --quiet

# Upload an image from local
from google.colab import files
uploaded = files.upload()

# Import required libraries
import face_recognition
from PIL import Image
import torchvision.transforms as transforms
import torch
import io
import matplotlib.pyplot as plt

# Automatically get uploaded filename
input_path = list(uploaded.keys())[0]

# Load image
img = face_recognition.load_image_file(input_path)
face_locations = face_recognition.face_locations(img)

# Check for face
if not face_locations:
    raise ValueError(" No face detected! Try uploading a clear image.")

# Crop the first face found
top, right, bottom, left = face_locations[0]
face_crop = img[top:bottom, left:right]
face_image = Image.fromarray(face_crop)

# Optional: show cropped face
plt.imshow(face_image)
plt.title("Detected & Cropped Face")
plt.axis("off")
plt.show()

# Preprocess for e4e
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3),
])

img_tensor = transform(face_image).unsqueeze(0).cuda()

# Encode into latent space
with torch.no_grad():
    latent = net.encoder(img_tensor)

print("✅ Face uploaded, cropped, and encoded into latent space!")


In [ ]:
from torchvision.utils import save_image
import torchvision.transforms.functional as TF

# Decode latent into image
with torch.no_grad():
    generated_image, _ = net.decoder(latent)

# Normalize from [-1, 1] to [0, 1]
generated_image = (generated_image + 1) / 2

# Save and display
save_image(generated_image, 'reconstructed_face.png')

from IPython.display import Image as IPyImage, display
display(IPyImage('reconstructed_face.png'))



# Version 1, This will give you a new face every 1 second, run this and copy the code into the script from here https://github.com/markduink/Doppelganger_videocall

In [ ]:
# 🔧 Install required packages
!pip install flask flask_cors pyngrok face-recognition

from flask import Flask, request, jsonify, make_response
from flask_cors import CORS
import numpy as np
import base64
from io import BytesIO
from PIL import Image
import torch
import torchvision.transforms as transforms
import face_recognition
import threading
from pyngrok import ngrok

# Initialize Flask app
app = Flask(__name__)
CORS(app, origins=["https://markduink.github.io"])  # Only allow GitHub Pages

# 🔄 Ensure model `net` is already loaded
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

def apply_e4e(image):
    img_np = np.array(image)
    face_locations = face_recognition.face_locations(img_np)
    if not face_locations:
        raise ValueError("No face detected.")

    top, right, bottom, left = face_locations[0]
    face_crop = image.crop((left, top, right, bottom))
    aligned = transform(face_crop).unsqueeze(0).cuda()

    with torch.no_grad():
        latent = net.encoder(aligned)
        generated_image, _ = net.decoder(latent)

    generated_image = (generated_image + 1) / 2
    if generated_image.dim() == 4:
        generated_image = generated_image[0]
    generated_image = generated_image.permute(1, 2, 0).cpu().numpy()
    generated_image = (generated_image * 255).astype(np.uint8)
    return Image.fromarray(generated_image)

@app.route('/process', methods=['POST', 'OPTIONS'])
def process_image():
    if request.method == 'OPTIONS':
        response = make_response()
        response.headers['Access-Control-Allow-Origin'] = 'https://markduink.github.io'
        response.headers['Access-Control-Allow-Headers'] = 'Content-Type'
        response.headers['Access-Control-Allow-Methods'] = 'POST, OPTIONS'
        return response

    try:
        data = request.json['image']
        image_bytes = base64.b64decode(data)
        image = Image.open(BytesIO(image_bytes)).convert('RGB')
        transformed = apply_e4e(image)

        buffer = BytesIO()
        transformed.save(buffer, format="JPEG")
        encoded_result = base64.b64encode(buffer.getvalue()).decode('utf-8')

        response = make_response(jsonify({'image': encoded_result}))
        response.headers['Access-Control-Allow-Origin'] = 'https://markduink.github.io'
        return response

    except Exception as e:
        response = make_response(jsonify({'error': str(e)}), 400)
        response.headers['Access-Control-Allow-Origin'] = 'https://markduink.github.io'
        return response

# Run Flask in thread
def run_flask():
    try:
        app.run(host='0.0.0.0', port=5000)
    except OSError:
        print("⚠️ Port 5000 already in use, skipping.")

threading.Thread(target=run_flask, daemon=True).start()

# Expose via ngrok
!pkill -f ngrok
!ngrok authtoken 2ubEflxgxaWcUp773o7osK572FR_85fsgXR1Zb79KekWMAp3D  # Replace with your actual ngrok token
public_url = ngrok.connect(5000).public_url
print("🚀 Your backend URL is:", public_url + "/process")


# Version 2, This will loop in the latent space giving you a animated face doppelganger , run this and copy the link provided into the java script from here https://github.com/markduink/Doppelganger_videocall

In [ ]:
# Install required packages
!pip install flask flask_cors pyngrok face-recognition

from flask import Flask, request, jsonify, make_response
from flask_cors import CORS
import numpy as np
import base64
from io import BytesIO
from PIL import Image
import torch
import torchvision.transforms as transforms
import face_recognition
import threading
from pyngrok import ngrok
import time

# Initialize Flask app
app = Flask(__name__)
CORS(app, origins=["https://markduink.github.io"])

# Ensure model `net` is already loaded
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# Interpolation state (preserved across frames)
interpolation_state = {
    "latent_start": None,
    "latent_end": None,
    "alpha": 0.0,
    "step": 0.05
}

def interpolate_latents(start, end, alpha):
    return (1 - alpha) * start + alpha * end

def apply_e4e_looped(image):
    img_np = np.array(image)
    face_locations = face_recognition.face_locations(img_np)
    if not face_locations:
        raise ValueError("No face detected.")

    top, right, bottom, left = face_locations[0]
    face_crop = image.crop((left, top, right, bottom))
    aligned = transform(face_crop).unsqueeze(0).cuda()

    # Update interpolation
    with torch.no_grad():
        if interpolation_state["latent_start"] is None:
            interpolation_state["latent_start"] = net.encoder(aligned)
            interpolation_state["latent_end"] = torch.randn_like(interpolation_state["latent_start"])

        alpha = interpolation_state["alpha"]
        interpolated_latent = interpolate_latents(interpolation_state["latent_start"], interpolation_state["latent_end"], alpha)

        # Advance alpha
        interpolation_state["alpha"] += interpolation_state["step"]
        if interpolation_state["alpha"] >= 1.0:
            interpolation_state["alpha"] = 0.0
            interpolation_state["latent_start"] = interpolated_latent
            interpolation_state["latent_end"] = torch.randn_like(interpolated_latent)

        generated_image, _ = net.decoder(interpolated_latent)

    generated_image = (generated_image + 1) / 2
    if generated_image.dim() == 4:
        generated_image = generated_image[0]
    generated_image = generated_image.permute(1, 2, 0).cpu().numpy()
    generated_image = (generated_image * 255).astype(np.uint8)
    return Image.fromarray(generated_image)

@app.route('/process', methods=['POST', 'OPTIONS'])
def process_image():
    if request.method == 'OPTIONS':
        response = make_response()
        response.headers['Access-Control-Allow-Origin'] = 'https://markduink.github.io'
        response.headers['Access-Control-Allow-Headers'] = 'Content-Type'
        response.headers['Access-Control-Allow-Methods'] = 'POST, OPTIONS'
        return response

    try:
        data = request.json['image']
        image_bytes = base64.b64decode(data)
        image = Image.open(BytesIO(image_bytes)).convert('RGB')
        transformed = apply_e4e_looped(image)

        buffer = BytesIO()
        transformed.save(buffer, format="JPEG")
        encoded_result = base64.b64encode(buffer.getvalue()).decode('utf-8')

        response = make_response(jsonify({'image': encoded_result}))
        response.headers['Access-Control-Allow-Origin'] = 'https://markduink.github.io'
        return response

    except Exception as e:
        response = make_response(jsonify({'error': str(e)}), 400)
        response.headers['Access-Control-Allow-Origin'] = 'https://markduink.github.io'
        return response

# Run Flask in thread
def run_flask():
    try:
        app.run(host='0.0.0.0', port=5000)
    except OSError:
        print("⚠️ Port 5000 already in use, skipping.")

threading.Thread(target=run_flask, daemon=True).start()

# Expose via ngrok
!pkill -f ngrok
!ngrok authtoken 2ubEflxgxaWcUp773o7osK572FR_85fsgXR1Zb79KekWMAp3D
public_url = ngrok.connect(5000).public_url
print("🚀 Your backend URL is:", public_url + "/process")
